In [1]:
import pandas as pd
import pyreadr
import numpy as np
from tqdm import tqdm

In [2]:
path = '/Users/annieulichney/Library/CloudStorage/Box-Box/Deforestation_MachineLearning/Data/cand_prefeito.Rdata'
pathvote = '/Users/annieulichney/Library/CloudStorage/Box-Box/Deforestation_MachineLearning/Data/cand_prefeito_vote.Rdata'

df = pyreadr.read_r(path)['cand']
vote = pyreadr.read_r(pathvote)['cand']



In [3]:
#for each mun and each year want winning candidate, bio info, margin etc, 

#votes, total votes, pct votes, political party, incumbent runs, incumbent wins, seat limit

In [4]:
vote.head()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
0,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PSD,41,NaN,NaN,nao eleito,2388
1,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSTU,16,NaN,NaN,nao eleito,1713
2,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PAN,26,NaN,NaN,nao eleito,15721
3,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PCO,29,NaN,NaN,nao eleito,705
4,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PGT,30,NaN,NaN,nao eleito,989


In [5]:
keep = vote[vote.resultado == 'eleito'][['ano', 'id_municipio', 'id_candidato_bd', 'sigla_partido' ,'votos']]
keep.columns = ['ano', 'id_municipio', 'id_candidato_bd', 'sigla_partido', 'votos_winner']

#total votes for that municipio that year
keep = pd.merge(keep, vote[['ano', 'id_municipio', 'votos']].groupby(['ano', 'id_municipio']).agg('sum').reset_index(), how = 'left', left_on = ['ano', 'id_municipio'], right_on = ['ano', 'id_municipio'])
keep.head()

,ano,id_municipio,id_candidato_bd,sigla_partido,votos_winner,votos
0,2000,2708907,1265591,PSD,3147,4882
1,2000,1302553,19147,PST,1455,5658
2,2000,2903300,1071793,PAN,2135,3598
3,2000,2906907,962907,PSD,4614,8760
4,2000,2908606,1523132,PST,3643,8872


In [6]:
keep['incumbant_run'] = np.zeros(keep.shape[0])
keep['incumbant_win'] = np.zeros(keep.shape[0])
keep['term_lim_seat'] = np.zeros(keep.shape[0])
keep['margin'] = np.zeros(keep.shape[0])

In [7]:
#first find does incumbant run and does incumbant win? 
missing_incumbant = []

for i in tqdm(range(vote.shape[0])):
    row = vote.iloc[i]
    row

    #did they win? 
    if row.resultado == 'eleito':
        #check 4 years from current year for year+4 and id match
        incumbant_run = vote[(vote.ano == row.ano + 4) & (vote.id_candidato_bd == row.id_candidato_bd) & (vote.id_municipio == row.id_municipio)].shape[0]
        
       
        #if they run in 4 years, check if they win 
        if incumbant_run == 1:
            if vote[(vote.ano == row.ano + 4) & (vote.id_candidato_bd == row.id_candidato_bd)].shape[0] == 0:
                print('Missing data with :' + str(i))

            else:
                incumbant_win = int(vote[(vote.ano == row.ano + 4) & (vote.id_candidato_bd == row.id_candidato_bd)].resultado == 'eleito')
        
        
        #if they do not run in 4 years, win is 0
        if incumbant_run==0:
            incumbant_win = 0
        
        #stop if year is 2020 because 2024 not in dataset
        if row.ano == 2020:
            continue
        
        #update incumbent run and incumbent win for each row
        if keep[(keep.ano == row.ano+4)&(keep.id_municipio == row.id_municipio)].shape[0] == 0:
            #print('Missing row at :' + str(i) + 'year = ' + str(row.ano+4) + ' munic = ' + str(row.id_municipio))
            missing_incumbant.append([i, row.ano+4,row.id_municipio])
        else:
            keep.loc[keep[(keep.ano == row.ano+4)&(keep.id_municipio == row.id_municipio)].index[0], 'incumbant_run'] = incumbant_run
            keep.loc[keep[(keep.ano == row.ano+4)&(keep.id_municipio == row.id_municipio)].index[0], 'incumbant_win'] = incumbant_win




100%|██████████| 96838/96838 [14:21<00:00, 112.37it/s] 


In [8]:
keep['term_lim_seat'] = np.zeros(keep.shape[0])
missing_lim = []

for i in tqdm(range(vote.shape[0])):
    row = vote.iloc[i]
    
    #first check this person won
    if row.resultado == 'eleito':
        
        #check that same person won in same muni in 4 years
        if vote[(vote.id_municipio == row.id_municipio) & (vote.resultado == 'eleito') & (vote.id_candidato_bd == row.id_candidato_bd) & (vote.ano == row.ano + 4)].shape[0] != 0:
            
            if keep[(keep.ano == row.ano + 8) & (keep.id_municipio == row.id_municipio)].shape[0] == 0:
                missing_lim.append((row.ano, row.id_municipio, row.id_candidato_bd))
           
            elif keep[(keep.ano == row.ano + 8) & (keep.id_municipio == row.id_municipio)].shape[0] == 1:
                keep.loc[keep[(keep.ano == row.ano + 8) & (keep.id_municipio == row.id_municipio)].index[0], 'term_lim_seat'] = 1



        
    

100%|██████████| 96838/96838 [11:46<00:00, 137.15it/s] 


In [9]:
# #now do if the seat is term limited: 
# missing = []

# for i in tqdm(range(vote.shape[0])):
#     row = vote.iloc[i]
    
#     temp_frame = vote[(vote.id_candidato_bd == row.id_candidato_bd)&(vote.resultado == 'eleito')]
#     temp_frame = temp_frame.sort_values('ano').reset_index()
    
#     for index, row2 in temp_frame.iterrows():
#         if temp_frame[temp_frame.ano == row2.ano+4].shape[0] != 0:
            
#             if keep[(keep.ano == row2.ano + 8) & (keep.id_municipio == row2.id_municipio)].shape[0] == 0:
#                 missing.append([i,row2.ano+8,row2.id_municipio])
#                 #missing.append('Missing at ' + str(i) + ' year = ' + str(row.ano) + str(' munic = ') + str(row.id_municipio))
                                 
#             else:
#                 keep.loc[keep[(keep.ano == row2.ano + 8) & (keep.id_municipio == row2.id_municipio)].index[0], 'term_lim_seat'] = 1


    
    

In [10]:
#now get margin of victory


vote_missing = []
for i in tqdm(range(keep.shape[0])):
    row = keep.iloc[i]
    
    if vote[(vote.ano == row.ano) & (vote.id_municipio == row.id_municipio)].shape[0] == 0:
        vote_missing.append(i, row.ano, row.id_municipio)
    else:
        vote_sort = sorted(vote[(vote.ano == row.ano) & (vote.id_municipio == row.id_municipio)].votos, reverse = True)
        vote_sort.append(0)
        margin = vote_sort[0] - vote_sort[1]
        
    if keep[(keep.ano == row.ano) & (keep.id_municipio == row.id_municipio)].shape[0] != 0:
        keep.loc[keep[(keep.ano == row.ano) & (keep.id_municipio == row.id_municipio)].index[0], 'margin'] = margin
        
        
        

100%|██████████| 33756/33756 [11:05<00:00, 50.70it/s]


In [11]:
keep['votos_pct'] = keep.votos_winner/keep.votos

In [16]:
keep2 = pd.merge(keep, df[['id_candidato_bd', 'genero', 'ocupacao', 'instrucao', 'ano', 'idade']], how = 'left', left_on = ['id_candidato_bd', 'ano'], right_on = ['id_candidato_bd', 'ano'])




In [17]:
keep2.head()

,ano,id_municipio,id_candidato_bd,sigla_partido,votos_winner,votos,incumbant_run,incumbant_win,term_lim_seat,margin,votos_pct,genero,ocupacao,instrucao,idade
0,2000,2708907,1265591,PSD,3147,4882,0.0,0.0,0.0,1412.0,0.644613,masculino,proprietario de estabelecimento comercial,ensino medio completo,45
1,2000,1302553,19147,PST,1455,5658,0.0,0.0,0.0,155.0,0.257158,masculino,medico,ensino superior completo,31
2,2000,2903300,1071793,PAN,2135,3598,0.0,0.0,0.0,708.0,0.593385,masculino,outros,ensino superior completo,51
3,2000,2906907,962907,PSD,4614,8760,0.0,0.0,0.0,640.0,0.526712,masculino,outros,ensino fundamental incompleto,58
4,2000,2908606,1523132,PST,3643,8872,0.0,0.0,0.0,1015.0,0.410618,masculino,proprietario de estabelecimento comercial,ensino fundamental completo,42


In [18]:
keep2.shape

(125491, 15)

In [96]:
#investigate where people ran in different munis for kathryn

# not_same = []

# for i in tqdm(range(vote.shape[0])):
#     row = vote.iloc[i]
#     if row.resultado == 'eleito':
#         if len((np.unique(vote[(vote.id_candidato_bd == row.id_candidato_bd)].id_municipio))) != 1:
#             not_same.append([row.id_candidato_bd, np.unique(vote[(vote.id_candidato_bd == row.id_candidato_bd)].id_municipio)])
    

# not_same_df = pd.DataFrame(not_same, columns = ['id_candidato_bd', 'munis'])
# not_same_df = not_same_df[not_same_df['id_candidato_bd'].notna()]
# not_same_df = not_same_df.drop_duplicates(subset='id_candidato_bd', keep="first")
# not_same_df.to_csv('different_munis_election.csv')

100%|██████████| 96838/96838 [03:31<00:00, 457.34it/s] 


In [284]:
#pd.merge(keep, vote[['ano', 'id_municipio', 'votos']].groupby(['ano', 'id_municipio']).agg('sum').reset_index(), how = 'left', left_on = ['ano', 'id_municipio'], right_on = ['ano', 'id_municipio'])



In [230]:
keep2 = pd.merge(keep, df[['id_candidato_bd', 'genero', 'ocupacao', 'instrucao']], how = 'left', left_on = ['id_candidato_bd', 'ano'], right_on = ['id_candidato_bd', 'ano'])


In [231]:
keep.to_csv('election_munic.csv')

In [ ]:
#

In [237]:
missing_df = pd.DataFrame(missing_incumbant, columns = ['index', 'ano', 'id_candidato_bd'])

In [238]:
votes_missing_df = pd.DataFrame(missing, columns = ['index', 'ano', 'id_candidato_bd'])

In [244]:
keep.head()

,ano,id_municipio,id_candidato_bd,sigla_partido,votos_winner,votos,incumbant_run,incumbant_win,term_lim_seat,margin,votos_pct,genero,ocupacao,instrucao
0,2000,2708907,1265591,PSD,3147,4882,0.0,0.0,0.0,1412.0,0.644613,masculino,proprietario de estabelecimento comercial,ensino medio completo
1,2000,1302553,19147,PST,1455,5658,0.0,0.0,0.0,155.0,0.257158,masculino,medico,ensino superior completo
2,2000,1302553,19147,PST,1455,5658,0.0,0.0,0.0,155.0,0.257158,masculino,medico,ensino superior completo
3,2000,2903300,1071793,PAN,2135,3598,0.0,0.0,0.0,708.0,0.593385,masculino,outros,ensino superior completo
4,2000,2903300,1071793,PAN,2135,3598,0.0,0.0,0.0,708.0,0.593385,masculino,aposentado (exceto servidor publico),ensino superior completo


In [243]:
vote[]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
0,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PSD,41,NaN,NaN,nao eleito,2388
1,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSTU,16,NaN,NaN,nao eleito,1713
2,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PAN,26,NaN,NaN,nao eleito,15721
3,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PCO,29,NaN,NaN,nao eleito,705
4,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PGT,30,NaN,NaN,nao eleito,989


In [246]:
vote_missing

[]